In [ ]:
%cd ../..
%pip install .
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'pedestrian'
) as pedestrians

select *
from pedestrians as t1
join Cameras as cam on t1.cameraId = Cameras.id
where
  t1.heading < 180 and t1.heading > -180 and
  (contained(t1.traj, road_segment("roads"), cam.timestamp) or
   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135
"""

In [ ]:
### Prepare the world for queries ###
from apperception.new_world import *
# directly ingest the mini dataset and boston roadnetwork if needed
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

# World.db.reset()
name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.predicate(lambda obj: obj.object_type == 'vehicle.pedestrian')
def pred(obj1, cam):
    return (
        obj1.heading < 180 and obj1.heading > -180 and
        (F.contained(obj1.traj, F.road_segment("roads"), cam.timestamp) or
            F.contained(obj1.traj, F.road_segment("intersections")) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam, cam.timestamp) < 135
    )
world = world.predicate(pred)